# Sentiment Reputation Monitoring - Esplorazione e Report

Questo notebook documenta l'approccio, i risultati su campioni e idee di miglioramento per il sistema di monitoraggio della reputazione online.


In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Configurazione
API_BASE_URL = "http://localhost:8000"
SAMPLES_FILE = Path("../src/data/samples.jsonl")


## 1. Caricamento Campioni

Carichiamo i campioni etichettati dal file `samples.jsonl`.


In [ ]:
# Carica samples
samples = []
with open(SAMPLES_FILE, 'r') as f:
    for line in f:
        if line.strip():
            samples.append(json.loads(line))

df_samples = pd.DataFrame(samples)
print(f"Caricati {len(df_samples)} campioni")
print(f"\nDistribuzione label originale:")
print(df_samples['label'].value_counts())


## 2. Test API Locale

Verifichiamo che l'API sia disponibile e testiamo le predizioni sui campioni.


In [ ]:
# Health check
try:
    response = requests.get(f"{API_BASE_URL}/health")
    print("Health check:", response.json())
except Exception as e:
    print(f"API non disponibile. Avvia l'API con: uvicorn src.app.main:app --reload")
    print(f"Errore: {e}")


In [ ]:
# Test predizioni sui campioni
predictions = []
for idx, sample in enumerate(df_samples.itertuples()):
    try:
        response = requests.post(
            f"{API_BASE_URL}/predict",
            json={"text": sample.text},
            timeout=10
        )
        if response.status_code == 200:
            pred = response.json()
            predictions.append({
                "text": sample.text[:50],
                "expected_label": sample.label,
                "predicted_label": pred["label"],
                "score": pred["score"],
                "match": sample.label == pred["label"]
            })
        else:
            print(f"Errore per sample {idx}: {response.status_code}")
    except Exception as e:
        print(f"Errore per sample {idx}: {e}")

df_predictions = pd.DataFrame(predictions)
if len(df_predictions) > 0:
    print(f"\nRisultati predizioni:")
    print(df_predictions[["expected_label", "predicted_label", "match"]])
    accuracy = df_predictions["match"].mean()
    print(f"\nAccuracy: {accuracy:.2%}")
else:
    print("Nessuna predizione disponibile. Avvia l'API prima.")


## 3. Distribuzione Sentiment

Analizziamo la distribuzione delle predizioni di sentiment.


In [ ]:
if len(df_predictions) > 0:
    pred_dist = df_predictions["predicted_label"].value_counts()
    print("Distribuzione predizioni:")
    print(pred_dist)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    expected_dist = df_samples["label"].value_counts()
    pred_dist_plot = df_predictions["predicted_label"].value_counts()
    
    axes[0].bar(expected_dist.index, expected_dist.values, alpha=0.7, label="Attesa")
    axes[0].set_title("Distribuzione Label Attesa")
    axes[0].set_ylabel("Frequenza")
    
    axes[1].bar(pred_dist_plot.index, pred_dist_plot.values, alpha=0.7, color="orange", label="Predetta")
    axes[1].set_title("Distribuzione Label Predetta")
    axes[1].set_ylabel("Frequenza")
    
    plt.tight_layout()
    plt.show()
else:
    print("Nessun dato disponibile per visualizzazione")


## 4. Calcolo Drift (KL Divergence)

Calcoliamo la KL divergence tra distribuzione di riferimento (baseline) e distribuzione corrente per rilevare concept drift.


In [ ]:
# Importa funzione di drift
import sys
sys.path.append('../src')
from utils.drift import kl_divergence, compute_reference_distribution, windowed_label_distribution

if len(df_predictions) > 0:
    baseline_dist = compute_reference_distribution(df_samples["label"].tolist())
    print("Distribuzione baseline (riferimento):")
    print(baseline_dist)
    
    current_labels = df_predictions["predicted_label"].tolist()
    current_dist = compute_reference_distribution(current_labels)
    print("\nDistribuzione corrente (predizioni):")
    print(current_dist)
    
    kl_value = kl_divergence(baseline_dist, current_dist)
    print(f"\nKL Divergence: {kl_value:.4f}")
    print(f"Interpretazione: {'Drift rilevato' if kl_value > 0.1 else 'Distribuzione stabile'}")
else:
    print("Nessun dato disponibile per calcolo drift")


## 5. Analisi Finestre Mobili

Simuliamo il calcolo di drift su finestre mobili (windowed analysis).


In [ ]:
if len(df_predictions) > 0:
    labels_series = df_predictions["predicted_label"].tolist()
    window_size = 5
    
    window_distributions = windowed_label_distribution(labels_series, window_size)
    
    kl_values = []
    for dist in window_distributions:
        kl = kl_divergence(baseline_dist, dist)
        kl_values.append(kl)
    
    plt.figure(figsize=(10, 4))
    plt.plot(kl_values, marker='o')
    plt.axhline(y=0.1, color='r', linestyle='--', label='Soglia drift (0.1)')
    plt.xlabel('Finestra Temporale')
    plt.ylabel('KL Divergence')
    plt.title('Drift Detection su Finestre Mobili')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Media KL divergence: {np.mean(kl_values):.4f}")
    print(f"Max KL divergence: {np.max(kl_values):.4f}")
else:
    print("Nessun dato disponibile")


## Conclusioni e Sviluppi Futuri

### Risultati
- Modello utilizzato: `cardiffnlp/twitter-roberta-base-sentiment-latest`
- Accurtezza su campioni di test: da valutare in base ai risultati sopra
- Drift detection: implementato con KL divergence

### Miglioramenti Possibili

- Alerting automatico quando KL divergence supera soglia
- Fine-tuning del modello su dominio specifico se necessario
- Integrazione con API social per raccolta dati automatica
- Rate limiting per proteggere l'API da sovraccarico
- Caching delle predizioni per testi frequenti
- Retraining periodico con nuovi dati per mantenere accuratezza
